In [1]:
import requests
import datetime
import pandas as pd
import json
import os

### Readme

This script contains several constituent functions which are run together in the bottom cell. Make sure to run all cells prior to running the bottom. There is an explanation above each cell as to what that function does and what inputs it takes. In summary, the whole thing is designed to: 

    1) log into the Aeroqual Cloud API
    2) check for first date data are needed or missing for the monitor
    3) request data from date identified in step 2 up to yesterday in one-week intervals
    4) concatenate data for each monitor and write to csv
    5) concatenate individual monitors into one master dataset

Login to aeroqual cloud. Requires inputs of username and password. This requires either Lee Ann or Boris' credentials to login to the cloud

In [2]:
def create_aqc_login(username, password):
    
    credentials = {'UserName': username, 'Password': password}
    
    aqc_login = requests.post('https://cloud.aeroqual.com/api/account/login/', data = credentials)
    
    print('Login status:', aqc_login)
    
    return(aqc_login)

Get list of available instruments. This will return all instruments with data available on the cloud for any period of time and any pollutants. 

In [3]:
def get_instruments_available(aqc_login):
    
    available_aqys = requests.get('https://cloud.aeroqual.com/api/instrument', cookies = aqc_login.cookies)
    
    print('Instrument list request status:', available_aqys)
    
    return(available_aqys.json())

All functions below this one are intended to work on one AQY at a time. Easiest use case is to iterate through the AQY list generated in get_instruments_available. This one gets the date the desired AQY was deployed as a first theoretically possible measurement.

In [4]:
def get_deployment_date(aqy_id):
    
    deployment_network = pd.read_csv('monitor_deployment.txt', sep = '\t')
    deployment_aqy = deployment_network[deployment_network['Monitor ID'] == aqy_id]
    
    deployment_date = deployment_aqy.iloc[0]['Deployment date']
    deployment_split = str.split(deployment_date, '-')
    
    deployment_date_dt = datetime.date(int(deployment_split[0]), int(deployment_split[1]), int(deployment_split[2]))
    print(aqy_id, 'deployed', deployment_date_dt.strftime('%Y-%m-%d'))
    
    return(deployment_date_dt)

Gets a list of all existing days in the dataset based on the existing filepath. Feeds into _find_date_to_start_ and is used to identify gaps in data.

In [5]:
def get_existing_dates_in_data(existing_file_path):
    
    existing_data = pd.read_csv(existing_file_path)
    existing_data['date_observed'] = pd.to_datetime(existing_data['Time'].str[0:11])
    
    existing_dates = pd.DataFrame(set(existing_data['date_observed'])).rename(columns = {0: 'date_observed'}).sort_values('date_observed')
    existing_dates['time_dif'] = existing_dates['date_observed'].diff()
    existing_dates = existing_dates.dropna()
        
    return(existing_dates)   

Uses existing days in data and set of all possible days to find first day not in observed data that should be. Returns this as first data gap.

In [6]:
def find_first_data_gap(existing_dates):
    
    dates_observed = existing_dates['date_observed']
    dates_observed_dt = set([datetime.date(date_observed.year, date_observed.month, date_observed.day) for date_observed in dates_observed])
    
    first_date_observed = existing_dates['date_observed'][0]
    yesterday = pd.Timestamp.today() - pd.Timedelta(value = 1, unit = 'D')
    dates_possible = pd.date_range(first_date_observed, yesterday)
    
    dates_possible_dt = set(dates_possible.date)
    dates_possible_dt.difference_update(dates_observed_dt)
    
    first_missing_date = min(dates_possible_dt)
    
    return(first_missing_date)

Checks for existing data in results folder for given AQY. If none exist, returns deployment date as first date of data needed. This is the parent function to _get deployment date, get_existing_dates_in_data, find_first_gap_in_data, and find_start_date_from_existing_.

In [7]:
def check_existing_get_start_date(aqy_id):

    existing_min_data_aqy = os.path.join(aqy_id + '_min.csv')
    
    existing_files_all_aqys = os.listdir('results/one_minute_data')
    
    deployment_date = get_deployment_date(aqy_id)
    
    if existing_min_data_aqy in existing_files_all_aqys:  
        
        existing_dates = get_existing_dates_in_data(os.path.join('results/one_minute_data', existing_min_data_aqy))
        
        first_data_gap = find_first_data_gap(existing_dates)                                         
        
        first_day_needed = max(deployment_date, first_data_gap)
        
        return(first_day_needed)
         
    else:
        
        return(deployment_date)

Gets all necessary query dates (start of week and end of week) based on start date returned in above functions. Formats in all needed formats.

In [8]:
def get_all_query_dates(start_date):
    
    yesterday = datetime.date.today() - datetime.timedelta(days = 1)
    
    desired_dates = {'start_date_dt': start_date, 
                     'end_date_dt': None, 
                     'start_date_str': None,
                     'end_date_str': None
                    }
    
    desired_dates['end_date_dt'] = min(desired_dates['start_date_dt'] + datetime.timedelta(days = 6), yesterday)
    
    desired_dates['start_date_str'] = desired_dates['start_date_dt'].strftime('%Y-%m-%d')
    desired_dates['end_date_str'] = desired_dates['end_date_dt'].strftime('%Y-%m-%d')
    
    return(desired_dates)

Requests AQY data between the start and end date given for the requested averaging time.

In [9]:
def get_aqy_data_for_week(aqc_login, aqy_id, start_date, end_date, raw_tf):
    
    request_url_id = 'https://api.cloud.aeroqual.com/V2/instruments/{}/data?averagingPeriod=1'.format(str(aqy_id.replace(' ', '%20')))
    request_url_dates = '&from={}%2000%3A00%3A00&to={}%2023%3A59%3A59'.format(start_date, end_date)
    request_url_rawTF = '&includeDiagnostics=true&rawValues={}&utc=false&includeSensorsWithNoData=true'.format(raw_tf)
    
    request_url = request_url_id + request_url_dates + request_url_rawTF
    
    aqy_data = requests.get(request_url, cookies = aqc_login.cookies)
    request_status = str(aqy_data)[-5:-2]
    
    if request_status != '200':
        print('Request status:', aqy_data)
        return(aqy_data.content)
    
    aqy_data_json = aqy_data.json()
    
    if 'Data' in aqy_data_json['Instruments'][0].keys():
        aqy_data_df = pd.json_normalize(aqy_data_json['Instruments'][0]['Data'])
    
        return(aqy_data_df)
    
    else:
        return(pd.DataFrame())

In [10]:
def initialize_results(aqy_id):
    
    existing_results_aqy = aqy_id + '_min.csv'
    existing_results_folder = 'results/one_minute_data'
    
    if existing_results_aqy in os.list.dir(existing_results_folder):
        existing_data_df = pd.read_csv(os.path.join(existing_results_folder, existing_results_aqy))
        
        return(existing_data_df)
    
    else:
        blank_df = pd.DataFrame()
        
        return(blank_df)

Starts querying the API on the start date generated through the first steps. Subsequently adds seven days onto query start date and end date until reaching yesterday. Appends requested data to existing data.

In [11]:
def get_aqy_data_for_full_deployment(aqc_login, aqy_id, raw_tf):
    
    date_to_start = check_existing_get_start_date(aqy_id)
    
    if date_to_start == 'Up-to-date': 
        return(print('Moving on to next AQY - data already up to date'))
    else:
        query_dates = get_all_query_dates(date_to_start)
    
    aqy_data_full = initialize_results(aqy_id)

    while query_dates['start_date_dt'] < datetime.date.today() - datetime.timedelta(days = 1):
        print(query_dates['start_date_dt'], query_dates['end_date_dt'])

        aqy_data = get_aqy_data_for_week(aqcloud_login, aqy_id, query_dates['start_date_str'], query_dates['end_date_str'], raw_tf)
    
        query_dates = get_query_dates(query_dates['start_date_dt'] + datetime.timedelta(days = 7))
        
        if aqy_data.shape[1] > 1:
            aqy_data_full = pd.concat([aqy_data_full, aqy_data])
    
    aqy_data_full['ID'] = aqy_id
    
    print('Data downloaded from cloud and appended to existing file')
    
    return(aqy_data_full)

### Put it Together

Script to collect Sacramento Co-location Data

In [15]:
def get_colocation_minute_data(raw_tf):
    aqcloud_login = create_aqc_login('lhill@psehealthyenergy.org', 'R1chmond@ir')

    aqys_available_list = get_instruments_available(aqcloud_login)
    aqys_available_list.remove('AQY BB-896')

    for aqy in aqys_available_list:

        colocation_start = datetime.date(2019, 7, 15)
        query_dates = get_all_query_dates(colocation_start)
        colocation_end = get_deployment_date(aqy)

        aqy_data_full = pd.DataFrame()

        while query_dates['start_date_dt'] < colocation_end:

            aqy_data = get_aqy_data_for_week(aqcloud_login, aqy, query_dates['start_date_str'], query_dates['end_date_str'], raw_tf)

            print('Requested data for', aqy, 'from', query_dates['start_date_dt'], 'to', query_dates['end_date_dt'])

            query_dates = get_all_query_dates(query_dates['start_date_dt'] + datetime.timedelta(days = 7))

            if aqy_data.shape[1] > 1:
                aqy_data_full = pd.concat([aqy_data_full, aqy_data])

        aqy_data_full['ID'] = aqy

        aqy_data_full.to_csv(os.path.join('results', 'sac_colocation', aqy + '_incNullTrue' + '_raw' + raw_tf + '.csv'), index = False)

In [16]:
get_colocation_minute_data('true')

Login status: <Response [200]>
Instrument list request status: <Response [200]>
AQY BB-607 deployed 2019-12-05
Requested data for AQY BB-607 from 2019-07-15 to 2019-07-21
Requested data for AQY BB-607 from 2019-07-22 to 2019-07-28
Requested data for AQY BB-607 from 2019-07-29 to 2019-08-04
Requested data for AQY BB-607 from 2019-08-05 to 2019-08-11
Requested data for AQY BB-607 from 2019-08-12 to 2019-08-18
Requested data for AQY BB-607 from 2019-08-19 to 2019-08-25
Requested data for AQY BB-607 from 2019-08-26 to 2019-09-01
Requested data for AQY BB-607 from 2019-09-02 to 2019-09-08
Requested data for AQY BB-607 from 2019-09-09 to 2019-09-15
Requested data for AQY BB-607 from 2019-09-16 to 2019-09-22
Requested data for AQY BB-607 from 2019-09-23 to 2019-09-29
Requested data for AQY BB-607 from 2019-09-30 to 2019-10-06
Requested data for AQY BB-607 from 2019-10-07 to 2019-10-13
Requested data for AQY BB-607 from 2019-10-14 to 2019-10-20
Requested data for AQY BB-607 from 2019-10-21 to 

Requested data for AQY BB-609 from 2019-11-11 to 2019-11-17
Requested data for AQY BB-609 from 2019-11-18 to 2019-11-24
Requested data for AQY BB-609 from 2019-11-25 to 2019-12-01
Requested data for AQY BB-609 from 2019-12-02 to 2019-12-08
Requested data for AQY BB-609 from 2019-12-09 to 2019-12-15
Requested data for AQY BB-609 from 2019-12-16 to 2019-12-22
AQY BB-610 deployed 2020-03-13
Requested data for AQY BB-610 from 2019-07-15 to 2019-07-21
Requested data for AQY BB-610 from 2019-07-22 to 2019-07-28
Requested data for AQY BB-610 from 2019-07-29 to 2019-08-04
Requested data for AQY BB-610 from 2019-08-05 to 2019-08-11
Requested data for AQY BB-610 from 2019-08-12 to 2019-08-18
Requested data for AQY BB-610 from 2019-08-19 to 2019-08-25
Requested data for AQY BB-610 from 2019-08-26 to 2019-09-01
Requested data for AQY BB-610 from 2019-09-02 to 2019-09-08
Requested data for AQY BB-610 from 2019-09-09 to 2019-09-15
Requested data for AQY BB-610 from 2019-09-16 to 2019-09-22
Requested

Requested data for AQY BB-635 from 2019-08-12 to 2019-08-18
Requested data for AQY BB-635 from 2019-08-19 to 2019-08-25
Requested data for AQY BB-635 from 2019-08-26 to 2019-09-01
Requested data for AQY BB-635 from 2019-09-02 to 2019-09-08
Requested data for AQY BB-635 from 2019-09-09 to 2019-09-15
Requested data for AQY BB-635 from 2019-09-16 to 2019-09-22
Requested data for AQY BB-635 from 2019-09-23 to 2019-09-29
Requested data for AQY BB-635 from 2019-09-30 to 2019-10-06
Requested data for AQY BB-635 from 2019-10-07 to 2019-10-13
Requested data for AQY BB-635 from 2019-10-14 to 2019-10-20
Requested data for AQY BB-635 from 2019-10-21 to 2019-10-27
Requested data for AQY BB-635 from 2019-10-28 to 2019-11-03
Requested data for AQY BB-635 from 2019-11-04 to 2019-11-10
Requested data for AQY BB-635 from 2019-11-11 to 2019-11-17
Requested data for AQY BB-635 from 2019-11-18 to 2019-11-24
Requested data for AQY BB-635 from 2019-11-25 to 2019-12-01
Requested data for AQY BB-635 from 2019-

Requested data for AQY BB-641 from 2019-09-09 to 2019-09-15
Requested data for AQY BB-641 from 2019-09-16 to 2019-09-22
Requested data for AQY BB-641 from 2019-09-23 to 2019-09-29
Requested data for AQY BB-641 from 2019-09-30 to 2019-10-06
Requested data for AQY BB-641 from 2019-10-07 to 2019-10-13
Requested data for AQY BB-641 from 2019-10-14 to 2019-10-20
Requested data for AQY BB-641 from 2019-10-21 to 2019-10-27
Requested data for AQY BB-641 from 2019-10-28 to 2019-11-03
Requested data for AQY BB-641 from 2019-11-04 to 2019-11-10
Requested data for AQY BB-641 from 2019-11-11 to 2019-11-17
Requested data for AQY BB-641 from 2019-11-18 to 2019-11-24
Requested data for AQY BB-641 from 2019-11-25 to 2019-12-01
Requested data for AQY BB-641 from 2019-12-02 to 2019-12-08
AQY BB-643 deployed 2019-12-06
Requested data for AQY BB-643 from 2019-07-15 to 2019-07-21
Requested data for AQY BB-643 from 2019-07-22 to 2019-07-28
Requested data for AQY BB-643 from 2019-07-29 to 2019-08-04
Requested

Requested data for AQY BB-646 from 2019-09-30 to 2019-10-06
Requested data for AQY BB-646 from 2019-10-07 to 2019-10-13
Requested data for AQY BB-646 from 2019-10-14 to 2019-10-20
Requested data for AQY BB-646 from 2019-10-21 to 2019-10-27
Requested data for AQY BB-646 from 2019-10-28 to 2019-11-03
Requested data for AQY BB-646 from 2019-11-04 to 2019-11-10
Requested data for AQY BB-646 from 2019-11-11 to 2019-11-17
Requested data for AQY BB-646 from 2019-11-18 to 2019-11-24
Requested data for AQY BB-646 from 2019-11-25 to 2019-12-01
Requested data for AQY BB-646 from 2019-12-02 to 2019-12-08
Requested data for AQY BB-646 from 2019-12-09 to 2019-12-15
Requested data for AQY BB-646 from 2019-12-16 to 2019-12-22
AQY BB-647 deployed 2019-12-16
Requested data for AQY BB-647 from 2019-07-15 to 2019-07-21
Requested data for AQY BB-647 from 2019-07-22 to 2019-07-28
Requested data for AQY BB-647 from 2019-07-29 to 2019-08-04
Requested data for AQY BB-647 from 2019-08-05 to 2019-08-11
Requested

...

Requested data for AQY BB-803 from 2019-11-04 to 2019-11-10
Requested data for AQY BB-803 from 2019-11-11 to 2019-11-17
Requested data for AQY BB-803 from 2019-11-18 to 2019-11-24
Requested data for AQY BB-803 from 2019-11-25 to 2019-12-01
Requested data for AQY BB-803 from 2019-12-02 to 2019-12-08
Requested data for AQY BB-803 from 2019-12-09 to 2019-12-15
Requested data for AQY BB-803 from 2019-12-16 to 2019-12-22
Requested data for AQY BB-803 from 2019-12-23 to 2019-12-29
Requested data for AQY BB-803 from 2019-12-30 to 2020-01-05
Requested data for AQY BB-803 from 2020-01-06 to 2020-01-12
Requested data for AQY BB-803 from 2020-01-13 to 2020-01-19
Requested data for AQY BB-803 from 2020-01-20 to 2020-01-26
Requested data for AQY BB-803 from 2020-01-27 to 2020-02-02
Requested data for AQY BB-803 from 2020-02-03 to 2020-02-09
Requested data for AQY BB-803 from 2020-02-10 to 2020-02-16
Requested data for AQY BB-803 from 2020-02-17 to 2020-02-23
Requested data for AQY BB-803 from 2020-

Requested data for AQY BB-806 from 2019-11-25 to 2019-12-01
Requested data for AQY BB-806 from 2019-12-02 to 2019-12-08
Requested data for AQY BB-806 from 2019-12-09 to 2019-12-15
Requested data for AQY BB-806 from 2019-12-16 to 2019-12-22
Requested data for AQY BB-806 from 2019-12-23 to 2019-12-29
Requested data for AQY BB-806 from 2019-12-30 to 2020-01-05
Requested data for AQY BB-806 from 2020-01-06 to 2020-01-12
Requested data for AQY BB-806 from 2020-01-13 to 2020-01-19
Requested data for AQY BB-806 from 2020-01-20 to 2020-01-26
Requested data for AQY BB-806 from 2020-01-27 to 2020-02-02
Requested data for AQY BB-806 from 2020-02-03 to 2020-02-09
Requested data for AQY BB-806 from 2020-02-10 to 2020-02-16
Requested data for AQY BB-806 from 2020-02-17 to 2020-02-23
Requested data for AQY BB-806 from 2020-02-24 to 2020-03-01
Requested data for AQY BB-806 from 2020-03-02 to 2020-03-08
Requested data for AQY BB-806 from 2020-03-09 to 2020-03-15
Requested data for AQY BB-806 from 2020-

Requested data for AQY BB-807 from 2020-03-02 to 2020-03-08
Requested data for AQY BB-807 from 2020-03-09 to 2020-03-15
Requested data for AQY BB-807 from 2020-03-16 to 2020-03-22
Requested data for AQY BB-807 from 2020-03-23 to 2020-03-29
Requested data for AQY BB-807 from 2020-03-30 to 2020-04-05
Requested data for AQY BB-807 from 2020-04-06 to 2020-04-12
Requested data for AQY BB-807 from 2020-04-13 to 2020-04-19
Requested data for AQY BB-807 from 2020-04-20 to 2020-04-26
Requested data for AQY BB-807 from 2020-04-27 to 2020-05-03
Requested data for AQY BB-807 from 2020-05-04 to 2020-05-10
Requested data for AQY BB-807 from 2020-05-11 to 2020-05-17
Requested data for AQY BB-807 from 2020-05-18 to 2020-05-24
Requested data for AQY BB-807 from 2020-05-25 to 2020-05-31
Requested data for AQY BB-807 from 2020-06-01 to 2020-06-07
Requested data for AQY BB-807 from 2020-06-08 to 2020-06-14
Requested data for AQY BB-807 from 2020-06-15 to 2020-06-21
Requested data for AQY BB-807 from 2020-

Requested data for AQY BB-811 from 2020-08-03 to 2020-08-09
Requested data for AQY BB-811 from 2020-08-10 to 2020-08-16
AQY BB-897 deployed 2020-08-12
Requested data for AQY BB-897 from 2019-07-15 to 2019-07-21
Requested data for AQY BB-897 from 2019-07-22 to 2019-07-28
Requested data for AQY BB-897 from 2019-07-29 to 2019-08-04
Requested data for AQY BB-897 from 2019-08-05 to 2019-08-11
Requested data for AQY BB-897 from 2019-08-12 to 2019-08-18
Requested data for AQY BB-897 from 2019-08-19 to 2019-08-25
Requested data for AQY BB-897 from 2019-08-26 to 2019-09-01
Requested data for AQY BB-897 from 2019-09-02 to 2019-09-08
Requested data for AQY BB-897 from 2019-09-09 to 2019-09-15
Requested data for AQY BB-897 from 2019-09-16 to 2019-09-22
Requested data for AQY BB-897 from 2019-09-23 to 2019-09-29
Requested data for AQY BB-897 from 2019-09-30 to 2019-10-06
Requested data for AQY BB-897 from 2019-10-07 to 2019-10-13
Requested data for AQY BB-897 from 2019-10-14 to 2019-10-20
Requested

Requested data for AQY BE-803A from 2019-12-23 to 2019-12-29
Requested data for AQY BE-803A from 2019-12-30 to 2020-01-05
Requested data for AQY BE-803A from 2020-01-06 to 2020-01-12


In [247]:
minute_files = os.listdir('results/predeploy_sacramento_data/raw')

all_aqy_data = pd.DataFrame()

for csv_file in minute_files:
    
    one_aqy = pd.read_csv(os.path.join('results', 'predeploy_sacramento_data', 'raw', csv_file))
    
    all_aqy_data = pd.concat([all_aqy_data, one_aqy])

In [248]:
all_aqy_data['Time'] = all_aqy_data['Time'].str.replace('T', ' ')

In [249]:
all_aqy_data = all_aqy_data[['ID', 'Time', 'Data.PM2.5', 'Data.NO2', 'Data.O3', 'Data.TEMP', 'Data.RH', 'Data.DP']].rename(columns = {'Data.PM2.5':'PM25_raw', 'Data.NO2':'NO2_raw', 'Data.O3':'O3_raw', 'Data.TEMP':'temp', 'Data.RH':'RH', 'Data.DP': 'DP'})

In [250]:
all_aqy_data

,ID,Time,PM25_raw,NO2_raw,O3_raw,temp,RH,DP
0,AQY BB-607,2019-07-17 13:34:00,3.8,NaN,NaN,34.98,32.8,16.2
1,AQY BB-607,2019-07-17 13:35:00,4.3,0.0,0.0,35.16,32.0,16.0
2,AQY BB-607,2019-07-17 13:36:00,4.8,0.0,0.0,35.28,31.5,15.8
3,AQY BB-607,2019-07-17 13:37:00,4.0,0.0,0.0,35.30,31.2,15.7
4,AQY BB-607,2019-07-17 13:38:00,4.1,0.0,0.0,35.14,30.9,15.4
...,...,...,...,...,...,...,...,...
96615,AQY BB-898,2019-12-02 14:35:00,2.7,NaN,NaN,27.69,45.8,14.9
96616,AQY BB-898,2019-12-02 14:36:00,2.8,NaN,NaN,27.60,46.4,15.1
96617,AQY BB-898,2019-12-02 14:37:00,2.8,NaN,NaN,27.62,46.9,15.3
96618,AQY BB-898,2019-12-02 14:38:00,2.8,NaN,NaN,27.70,47.2,15.4


In [161]:
aqcloud_login = create_aqc_login('blukanov@psehealthyenergy.org', 'R1chmond@ir')

Login status: <Response [200]>


In [ ]:
requests.get('htts://cloud.aeroqual')

In [17]:
aqcloud_login = create_aqc_login('lhill@psehealthyenergy.org', 'R1chmond@ir')

aqys_available_list = get_instruments_available(aqcloud_login)

aqy_data = get_aqy_data_for_full_deployment(aqcloud_login, 'AQY BB-800', '60')
file_name = os.path.join('results', 'one_minute_data', aqy + '_min.csv')
    
aqy_data.to_csv(file_name, index = False)

Login status: <Response [200]>
Instrument list request status: <Response [200]>


NameError: name 'check_existing_get_start_date' is not defined

In [134]:
all_files = pd.DataFrame()

for file in all_60_min:
    aqy_file = pd.read_csv(os.path.join('results', 'aqy_60_min', file))
    
    all_files = pd.concat([all_files, aqy_file])

In [137]:
all_files['Time'] = all_files['Time'].str.replace('T', ' ')

In [142]:
all_files[['ID', 'Time', 'NO2', 'O3', 'PM2.5', 'TEMP', 'RH', 'DP']].to_csv('results/all_aqy_60_min.csv', index = False)

#Finds the date to start querying the data based on whether there are missing days throughout the data and whether the most recent timestamp is yesterday. Only called within _check_existing_get_start_date_ if there are existing data. Otherwise, deployment date is used.

def find_start_date_from_existing(existing_file_path):
    
    one_day_difference = datetime.timedelta(days = 1)
    
    existing_dates = get_existing_dates_in_data(existing_file_path)  
    
    days_after_gaps = existing_dates[existing_dates['time_dif'] != one_day_difference]
    number_of_gaps = days_after_gaps.shape[0]
    day_after_last_timestamp = existing_dates['date_observed'][-1:] + one_day_difference
    
    yesterday = datetime.date.today() - one_day_difference
    
    if number_of_gaps == 0:
        
        if day_after_last_timestamp == yesterday:
        
            print('Go analyze data! This AQY is already up-to-date')

            return('Up-to-date')
    
        else:         
            print('No gaps in data, but not quite up to date. Requesting  data starting at:', str(day_after_last_timestamp))
        
            return(day_after_last_timestamp)
        
    else:
        
        missing_day = find_first_gap_in_data(days_after_gaps, existing_dates)
        print('Data missing starting at ' + str(missing_day) + ': requesting from this date forward')
            
        return(missing_day)

#Creates a list of unique dates in the data. Identifies days that are >1 day after the preceeding day and labels as a gap. Feeds into _find_date_to_start_ if gaps are identified.

def look_for_first_gap_in_existing(days_after_gaps, existing_dates):
    
    day_after_first_gap = days_after_gaps['date_observed'].iloc[0]
            
    all_data_before_first_gap = existing_dates[existing_dates['date_observed'] < day_after_first_gap]
            
    day_before_first_gap = all_data_before_first_gap['date_observed'].iloc[-1]
        
    missing_day = day_before_first_gap + one_day_difference
    
    return(missing_day)

def get_sensors_available(aqc_login, aqy_id):
    
    aqy_request_url = 'https://cloud.aeroqual.com/api/instrument/{}'.format(aqy_id)
    
    available_aqy_data = requests.get(aqy_request_url, cookies = aqc_login.cookies)
    
    if str(available_aqy_data)[-5:-2] != '200':
        print('Request for', aqy_id, 'status:', available_aqy_data)
        
        return(available_aqy_data)
    else:
        
        aqy_metadata = available_aqy_data.json()
        
        aqy_sensors = [sensor['name'] for sensor in aqy_metadata['sensors']]
        
        return(aqy_sensors)

def get_data_by_date_network(aqc_login, list_of_aqys, start_date, end_date, avg_time):
    
    print('Requesting network data for range:', start_date, end_date)
    
    network_data_df = pd.DataFrame()
    
    for aqy in list_of_aqys:
        
        aqy_data_json = get_data_by_aqy(aqc_login, aqy, start_date, end_date, avg_time)

        aqy_data_df = pd.json_normalize(aqy_data_json['data'])
        aqy_data_df['ID'] = aqy
        
        if aqy_data_df.shape[1] > 1:
            network_data_df = network_data_df.append(aqy_data_df)
    
    return(network_data_df)